In [1]:
pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cuspa

In [2]:
! pip install gradio==3.50.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.8 MB/s eta 0:00:00


In [3]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.8 MB/s eta 0:00:00


In [4]:
import whisper
import torch
from transformers import MarianMTModel, MarianTokenizer
import gradio as gr
from sentence_transformers import SentenceTransformer, util

In [5]:
# Load the Whisper model
whisper_model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 195MiB/s]


In [6]:
# Load Sentence Transformer for retrieval
retriever_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Dummy RAG document store
documents = {
    "doc1": "This is a document about artificial intelligence and machine learning.",
    "doc2": "This document describes the basics of deep learning and neural networks.",
    "doc3": "Here we discuss the impact of AI on different industries like healthcare, finance, and more.",
    "doc4": "The future of technology includes advancements in AI, quantum computing, and other fields.",
}

document_embeddings = retriever_model.encode(list(documents.values()), convert_to_tensor=True)


In [8]:

# Function to transcribe speech using Whisper
def transcribe_speech(file_path):
    result = whisper_model.transcribe(file_path)
    return result["text"]

# Function to detect language
def detect_language(audio_path):
    audio = whisper.load_audio(audio_path)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(whisper_model.device)
    _, probs = whisper_model.detect_language(mel)
    detected_language_code = max(probs, key=probs.get)

    # Map detected language codes to readable names
    language_mapping = {
        'en': 'English', 'es': 'Spanish', 'fr': 'French', 'de': 'German',
        'hi': 'Hindi', 'ja': 'Japanese', 'ru': 'Russian', 'ar': 'Arabic',
        'te': 'Telugu', 'zh': 'Chinese', 'pt': 'Portuguese'
    }

    return language_mapping.get(detected_language_code, detected_language_code).capitalize()



In [9]:
# Function to load translation model and tokenizer
def load_translation_model(language):
    model_name = {
        "Hindi": "Helsinki-NLP/opus-mt-en-hi",
        "Spanish": "Helsinki-NLP/opus-mt-en-es",
        "Japanese": "Helsinki-NLP/opus-mt-en-ja",
        "German": "Helsinki-NLP/opus-mt-en-de",
        "Russian": "Helsinki-NLP/opus-mt-en-ru",
        "Arabic": "Helsinki-NLP/opus-mt-en-ar",
        "Telugu": "Helsinki-NLP/opus-mt-en-te",
        "French": "Helsinki-NLP/opus-mt-en-fr",
        "Italian": "Helsinki-NLP/opus-mt-en-it",
        "English": "Helsinki-NLP/opus-mt-en-en"
    }

    if language not in model_name:
        raise ValueError(f"Translation model for {language} not available.")

    translation_model = MarianMTModel.from_pretrained(model_name[language])
    translation_tokenizer = MarianTokenizer.from_pretrained(model_name[language])
    return translation_model, translation_tokenizer



In [10]:
# Function to translate text
def translate_text(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    with torch.no_grad():
        translated_tokens = model.generate(**inputs)
    translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translation



In [11]:

# Function to retrieve a document based on the query
def retrieve_document(query):
    query_embedding = retriever_model.encode(query, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]
    top_score_idx = scores.argmax().item()
    return list(documents.values())[top_score_idx]



In [12]:
# Function to process the audio file and return transcriptions and translations
def process_audio(audio, target_language):
    # Transcribe the audio
    transcription = transcribe_speech(audio)

    # Detect the language spoken in the audio
    detected_language = detect_language(audio)

    if target_language == "English" and detected_language != "English":
      target_language = detected_language

    # Load the appropriate translation model
    translation_model, translation_tokenizer = load_translation_model(target_language)

    # Translate the transcribed text
    translated_text = translate_text(transcription, translation_model, translation_tokenizer)

    # Retrieve document based on the transcribed text
    retrieved_document = retrieve_document(transcription)

    return transcription, detected_language, translated_text, retrieved_document

In [15]:
# Create the Gradio interface
iface = gr.Interface(
    fn=process_audio,
    inputs=[
        gr.Audio(source="upload", type="filepath"),
        gr.Dropdown(["Hindi", "Spanish", "Japanese", "German", "Russian", "Arabic", "French", "Italian", "English"], label="Target Language")
    ],
    outputs=[
        gr.Textbox(label="Transcription"),
        gr.Textbox(label="Detected Language"),
        gr.Textbox(label="Translation"),
        gr.Textbox(label="Retrieved Document")
    ],
    title="Multilingual Speech Recognition, Translation, and Document Retrieval",
    description="Upload an audio file in any language, select a target language to get the transcription, translation, and retrieve a document based on the transcription."
)

# Launch the Gradio interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.50.2, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://55b3369b22c50180cb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
